## Labirinto Invisível

In [3]:
import pygame
import random

# Parâmetros do grid
GRID_SIZE = 10
TILE_SIZE = 50  # Tamanho de cada célula no grid
WIDTH, HEIGHT = GRID_SIZE * TILE_SIZE, GRID_SIZE * TILE_SIZE

# Tamanho da tela (aumentado para espaço extra ao lado direito do grid)
SCREEN_WIDTH = WIDTH + 300  # Adiciona 200px ao lado direito para informações
SCREEN_HEIGHT = HEIGHT

# Parâmetros das Recompensas
PONT_MIN = -50
DIAMANTE = 30
CHEGADA = 100
TENTATIVA = -1

# Cores
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
DIAMOND_COLOR = (0, 255, 255)

# Movimentos do jogador
MOVES = {
    pygame.K_w: (-1, 0),  # Cima
    pygame.K_s: (1, 0),   # Baixo
    pygame.K_a: (0, -1),  # Esquerda
    pygame.K_d: (0, 1),   # Direita
}

# Labirinto mestre definido manualmente (True = caminho, False = parede)
def criar_labirinto_mestre():
    return [
        [True, True, False, False, False, False, False, True, False, False],
        [False, True, True, False, False, True, True, True, True, False],
        [False, False, True, True, False, True, False, False, True, False],
        [False, False, False, True, True, True, False, False, True, False],
        [False, False, False, True, False, True, False, False, True, False],
        [True, True, True, True, False, True, True, True, True, False],
        [True, False, False, False, False, False, False, True, False, False],
        [True, True, True, True, True, True, False, True, False, False],
        [False, True, False, False, False, True, True, True, True, True],
        [True, True, True, True, False, False, False, False, False, True],
    ]



# Inicializar o labirinto descoberto
def criar_labirinto_descoberto():
    return [[None for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

# Recompensas (diamantes) no labirinto
def criar_recompensas():
    return [(1, 2), (4, 7), (7, 1), (8, 9)]  # Posições onde estão os diamantes

# Desenhar o labirinto na tela
def desenhar_labirinto(tela, labirinto_descoberto, jogador, objetivo, objetivo_descoberto, recompensas, recompensas_coletadas):
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            x = j * TILE_SIZE
            y = i * TILE_SIZE
            cell = labirinto_descoberto[i][j]

            if cell is None:
                pygame.draw.rect(tela, BLACK, (x, y, TILE_SIZE, TILE_SIZE))  # Não explorado
                
            elif cell:
                pygame.draw.rect(tela, WHITE, (x, y, TILE_SIZE, TILE_SIZE))  # Caminho

            else:
                pygame.draw.rect(tela, RED, (x, y, TILE_SIZE, TILE_SIZE))  # Parede

            # Desenhar o jogador como bolinha azul
            if (i, j) == jogador:
                pygame.draw.circle(tela, BLUE, (x + TILE_SIZE // 2, y + TILE_SIZE // 2), TILE_SIZE // 3)  # Bolinha no centro da célula

            # Desenhar recompensas (diamantes) se o caminho foi revelado
            if (i, j) in recompensas and cell:
                pygame.draw.rect(tela, DIAMOND_COLOR, (x, y, TILE_SIZE, TILE_SIZE))  # Diamante

            # Desenhar um triângulo para recompensas coletadas
            if (i, j) in recompensas_coletadas:
                pontos_triangulo = [
                    (x + TILE_SIZE // 2, y + TILE_SIZE // 4),  # Ponto superior
                    (x + TILE_SIZE // 4, y + 3 * TILE_SIZE // 4),  # Ponto inferior esquerdo
                    (x + 3 * TILE_SIZE // 4, y + 3 * TILE_SIZE // 4)  # Ponto inferior direito
                ]
                pygame.draw.polygon(tela, DIAMOND_COLOR, pontos_triangulo)  # Desenha o triângulo verde

            # Desenhar o objetivo (chegada) apenas se for descoberto
            if objetivo_descoberto and (i, j) == objetivo:
                pygame.draw.rect(tela, YELLOW, (x, y, TILE_SIZE, TILE_SIZE))  # Objetivo

            pygame.draw.rect(tela, BLACK, (x, y, TILE_SIZE, TILE_SIZE), 1)  # Borda da célula

# Verificar se o movimento é válido (dentro dos limites)
def dentro_do_grid(nova_pos):
    return 0 <= nova_pos[0] < GRID_SIZE and 0 <= nova_pos[1] < GRID_SIZE


# Função para desenhar um botão na tela
def desenhar_botao(tela, texto, posicao, tamanho, cor, cor_texto):
    fonte = pygame.font.Font(None, 36)
    retangulo_botao = pygame.Rect(posicao, tamanho)
    pygame.draw.rect(tela, cor, retangulo_botao)
    texto_botao = fonte.render(texto, True, cor_texto)
    texto_rect = texto_botao.get_rect(center=retangulo_botao.center)
    tela.blit(texto_botao, texto_rect)
    return retangulo_botao  # Retorna o retângulo do botão para verificar cliques


# Função para exibir mensagem de vitória ou derrota
def exibir_mensagem(tela, mensagem, cor):
    font = pygame.font.Font(None, 36)
    text = font.render(mensagem, True, cor)
    texto_rect = text.get_rect(topleft=(WIDTH + 10, 100))
    tela.blit(text, texto_rect)
    
    # Desenhar o botão de saída
    botao_sair = desenhar_botao(tela, "Sair", (WIDTH + 10, 200), (100, 50), GRAY, BLACK)
    pygame.display.flip()

    # Aguardar até o jogador clicar no botão de sair
    esperando = True
    while esperando:
        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                pygame.quit()
                exit()
            if evento.type == pygame.MOUSEBUTTONDOWN:
                # Verificar se o clique foi no botão de sair
                if botao_sair.collidepoint(evento.pos):
                    esperando = False  # Sai do loop se o botão for clicado
    
    
    
    #pygame.display.flip()
    # pygame.time.wait(3000)

    # Aguardar até o jogador pressionar alguma tecla ou clicar com o mouse
    # esperando = True
    # while esperando:
    #     for evento in pygame.event.get():
    #         if evento.type == pygame.QUIT:  # Permite fechar a janela diretamente
    #             pygame.quit()
    #             exit()
    #         if evento.type == pygame.KEYDOWN or evento.type == pygame.MOUSEBUTTONDOWN:
    #             esperando = False  # Sai do loop se qualquer tecla for pressionada ou botão for clicado




# Função principal do jogo
def jogar_labirinto():
    recompensas_coletadas = []
    pygame.init()
    tela = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("Labirinto Invisível - Manual com Pontuação")
    
    clock = pygame.time.Clock()
    rodando = True

    labirinto_mestre = criar_labirinto_mestre()
    labirinto_descoberto = criar_labirinto_descoberto()
    recompensas = criar_recompensas()
    jogador = (0, 0)  # Posição inicial do jogador
    objetivo = (9, 9)  # Posição do objetivo
    objetivo_descoberto = False  # Objetivo não revelado inicialmente
    labirinto_descoberto[jogador[0]][jogador[1]] = True  # Revela a posição inicial

    # Sistema de pontuação
    pontos = 0
    caminhos_explorados = 1
    caminhos_possiveis = sum(sum(row) for row in labirinto_mestre)
    
    #Fonte dos textos
    font = pygame.font.Font(None, 36)

    while rodando:
        tela.fill(WHITE)

        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                rodando = False

            if evento.type == pygame.KEYDOWN:
                if evento.key in MOVES:
                    move = MOVES[evento.key]
                    nova_pos = (jogador[0] + move[0], jogador[1] + move[1])

                    if dentro_do_grid(nova_pos):

                        # Verifica no labirinto mestre
                        if labirinto_mestre[nova_pos[0]][nova_pos[1]]:

                            # Só incrementa 'caminhos_explorados' se for um novo caminho não visitado
                            if labirinto_descoberto[nova_pos[0]][nova_pos[1]] is None:

                                caminhos_explorados += 1  # Incrementa ao explorar um caminho novo
                                
                            jogador = nova_pos
                            labirinto_descoberto[jogador[0]][jogador[1]] = True  # Revela caminho
                            pontos += TENTATIVA  # Perde 1 ponto por movimento

                            # Se o jogador pegar uma recompensa
                            if jogador in recompensas:
                                recompensas.remove(jogador)
                                recompensas_coletadas.append(jogador)  # Adiciona a posição à lista de recompensas coletadas
                                pontos += DIAMANTE  # Ganha pontos pela recompensa

                            # Se o jogador chegar na célula do objetivo, revela-o e ganha pontos
                            if jogador == objetivo:
                                objetivo_descoberto = True
                                pontos += CHEGADA  # Ganha 50 pontos por alcançar o objetivo
                        

                        else:
                            labirinto_descoberto[nova_pos[0]][nova_pos[1]] = False  # Revela parede
                            pontos += TENTATIVA  # Perde 1 ponto mesmo tentando se mover para a parede

                    else:
                        print("Movimento fora dos limites!")
        
        desenhar_labirinto(tela, labirinto_descoberto, jogador, objetivo, objetivo_descoberto, recompensas, recompensas_coletadas)

        # Exibir pontuação no lado direito fora do grid
        
        score_text = font.render(f"Pontuação: {pontos}", True, BLACK)
        tela.blit(score_text, (WIDTH + 10, 10))  # Exibe no lado direito

        # Percentual de caminhos explorados no labirinto
        caminhos_conhecidos = round(caminhos_explorados/caminhos_possiveis, 2)*100
        explore_text = font.render(f"Explorado: {caminhos_conhecidos}%", True, BLACK)
        tela.blit(explore_text, (WIDTH + 10, 50))  # Exibe no lado direito

        # Checa se o jogador perdeu por pontos negativos
        if pontos <= PONT_MIN:
            exibir_mensagem(tela, "Você perdeu!", RED)
            rodando = False

        # Checa se o jogador alcançou o objetivo
        if jogador == objetivo:
            exibir_mensagem(tela, "Você Saiu do Labirinto!", GREEN)
            rodando = False

        pygame.display.flip()
        clock.tick(30)

    pygame.quit()

if __name__ == "__main__":
    jogar_labirinto()
